In [28]:
import requests
from bs4 import BeautifulSoup

income_statement = {}

url = "https://finance.yahoo.com/quote/AAPL/financials?p=AAPL"
# url = "https://finance.yahoo.com/quote/AAPL/financials/?p=AAPL"

headers = {"User-Agent" : "Chrome/96.0.4664.110"}
headers = {"User-Agent" : "Chrome/127.0.6533.100"}

"""
This doesn't work
"""

# page = requests.get(url, headers=headers)
page = requests.get(url)
page_content = page.content
soup = BeautifulSoup(page_content,"html.parser")
tabl = soup.find_all("div" , {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
for t in tabl:
    rows = t.find_all("div" , {"class": "D(tbr) fi-row Bgc($hoverBgColor):h"})
    for row in rows:
        income_statement[row.get_text(separator="|").split("|")[0]] = row.get_text(separator="|").split("|")[1]


In [29]:
page

<Response [404]>

In [31]:
# !pip install selenium


In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
chrome_options.add_argument("--disable-gpu")  # Disable GPU usage
chrome_options.add_argument("--no-sandbox")  # Disable sandboxing

# Specify the path to the ChromeDriver
driver_path = 'chromedriver-win64/chromedriver-win64/chromedriver.exe'

# Initialize the WebDriver
# driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)
driver = webdriver.Chrome(options=chrome_options)

# Open the webpage
url = "https://finance.yahoo.com/quote/AAPL/financials/?p=AAPL"
driver.get(url)

# Wait for the page to load completely
time.sleep(5)

# Parse the page content with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Extract the financial data
data_rows = soup.find_all('div', class_='D(tbr)')

# Prepare a list to hold all the rows
financial_data = []

# Iterate over each row and extract the data
for row in data_rows:
    cols = row.find_all('div')
    cols_text = [col.text for col in cols]
    financial_data.append(cols_text)

# Close the WebDriver
driver.quit()

# Display the scraped data
for row in financial_data:
    print(row)

In [213]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
chrome_options.add_argument("--disable-gpu")  # Disable GPU usage
chrome_options.add_argument("--no-sandbox")  # Disable sandboxing

# Specify the path to the ChromeDriver
driver_path = 'chromedriver-win64/chromedriver-win64/chromedriver.exe'

# Initialize the WebDriver
# driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)
driver = webdriver.Chrome(options=chrome_options)

# tickers = ["AAPL","FB","CSCO","INFY.NS","3988.HK"]
tickers = ["AAPL","CSCO","INFY.NS","3988.HK"]
income_statatement_dict = {}
balance_sheet_dict = {}
cashflow_st_dict = {}


def _get_name(row):
    for i in row.split('|'):
        if len(i.strip()) > 0:
            return i.strip()

def _get_values(row):

    first = True 

    values = []
    
    for i in row.split('|'):
        if len(i.strip()) > 0 and first:
            first = False
            continue
            
        if len(i.strip()) > 0:
            values.append(i.strip())
            
    return values
    

_get_values(row.get_text(separator="|"))

for ticker in tickers:

    print(ticker)
    
    #scraping income statement
    url = "https://finance.yahoo.com/quote/{}/financials/?p={}".format(ticker,ticker)
    income_statement = {}
    table_title = {}
    
    driver.get(url)
    # Wait for the page to load completely
    time.sleep(5)

    # Parse the page content with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    tabl = soup.find_all("div" , {"class" : "table yf-1pgoo1f"})
    for t in tabl:
        heading = t.find_all("div" , {"class": "tableHeader yf-1pgoo1f"})
        for top_row in heading:
            # table_title[top_row.get_text(separator="|").split("|")[0]] = top_row.get_text(separator="|").split("|")[1:]
            table_title[_get_name(top_row.get_text(separator="|"))] = _get_values(top_row.get_text(separator="|"))
        rows = t.find_all("div" , {"class": "row lv-0 yf-1xjz32c"})
        for row in rows:
            income_statement[_get_name(row.get_text(separator="|"))] = _get_values(row.get_text(separator="|"))

    temp = pd.DataFrame(income_statement).T
    temp.columns = table_title["Breakdown"]
    income_statatement_dict[ticker] = temp      


    #scraping balance sheet statement
    url = "https://finance.yahoo.com/quote/{}/balance-sheet?p={}".format(ticker,ticker)
    balance_sheet = {}
    table_title = {}
    
    driver.get(url)
    # Wait for the page to load completely
    time.sleep(5)

    # Parse the page content with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    tabl = soup.find_all("div" , {"class" : "table yf-1pgoo1f"})
    for t in tabl:
        heading = t.find_all("div" , {"class": "tableHeader yf-1pgoo1f"})
        for top_row in heading:
            # table_title[top_row.get_text(separator="|").split("|")[0]] = top_row.get_text(separator="|").split("|")[1:]
            table_title[_get_name(top_row.get_text(separator="|"))] = _get_values(top_row.get_text(separator="|"))
        rows = t.find_all("div" , {"class": "row lv-0 yf-1xjz32c"})
        for row in rows:
            balance_sheet[_get_name(row.get_text(separator="|"))] = _get_values(row.get_text(separator="|"))

    temp = pd.DataFrame(balance_sheet).T
    temp.columns = table_title["Breakdown"]
    balance_sheet_dict[ticker] = temp
    
    #scraping cashflow statement
    url = "https://finance.yahoo.com/quote/{}/cash-flow?p={}".format(ticker,ticker)
    cashflow_statement = {}
    table_title = {}
    
    driver.get(url)
    # Wait for the page to load completely
    time.sleep(5)

    # Parse the page content with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    tabl = soup.find_all("div" , {"class" : "table yf-1pgoo1f"})
    for t in tabl:
        heading = t.find_all("div" , {"class": "tableHeader yf-1pgoo1f"})
        for top_row in heading:
            # table_title[top_row.get_text(separator="|").split("|")[0]] = top_row.get_text(separator="|").split("|")[1:]
            table_title[_get_name(top_row.get_text(separator="|"))] = _get_values(top_row.get_text(separator="|"))
        rows = t.find_all("div" , {"class": "row lv-0 yf-1xjz32c"})
        for row in rows:
            cashflow_statement[_get_name(row.get_text(separator="|"))] = _get_values(row.get_text(separator="|"))

    temp = pd.DataFrame(cashflow_statement).T
    temp.columns = table_title["Breakdown"]
    cashflow_st_dict[ticker] = temp 

AAPL
CSCO
INFY.NS
3988.HK


In [216]:
ticker, col

('AAPL', 'TTM')

In [212]:
pd.to_numeric(cashflow_st_dict[ticker][col].str.replace(r'[-|, ]', '', regex=True))

Operating Cash Flow                  110563000.0
Investing Cash Flow                    4448000.0
Financing Cash Flow                  108219000.0
End Cash Position                     33921000.0
Income Tax Paid Supplemental Data     28316000.0
Interest Paid Supplemental Data              NaN
Capital Expenditure                    8644000.0
Issuance of Capital Stock                    NaN
Issuance of Debt                             NaN
Repayment of Debt                      6672000.0
Repurchase of Capital Stock           81825000.0
Free Cash Flow                       101919000.0
Name: TTM, dtype: float64

In [217]:
balance_sheet_dict[ticker]

,9/30/2023,9/30/2022,9/30/2021,9/30/2020,9/30/2019
Total Assets,"352,583,000","352,755,000","351,002,000","323,888,000",--
Total Liabilities Net Minority Interest,"290,437,000","302,083,000","287,912,000","258,549,000",--
Total Equity Gross Minority Interest,"62,146,000","50,672,000","63,090,000","65,339,000",--
Total Capitalization,"157,427,000","149,631,000","172,196,000","164,006,000",--
Common Stock Equity,"62,146,000","50,672,000","63,090,000","65,339,000",--
Capital Lease Obligations,"12,842,000","12,411,000","11,803,000","9,842,000",--
Net Tangible Assets,"62,146,000","50,672,000","63,090,000","65,339,000",--
Working Capital,"-1,742,000","-18,577,000","9,355,000","38,321,000",--
Invested Capital,"173,234,000","170,741,000","187,809,000","177,775,000",--
Tangible Book Value,"62,146,000","50,672,000","63,090,000","65,339,000",--


In [218]:
#converting dataframe values to numeric
for ticker in tickers:
    for col in income_statatement_dict[ticker].columns:
        print(ticker, col)
        
        income_statatement_dict[ticker][col] = income_statatement_dict[ticker][col].str.replace(r'[-|, ]', '', regex=True)
        income_statatement_dict[ticker][col] = pd.to_numeric(income_statatement_dict[ticker][col])
        print('income_statatement_dict done')
        
        cashflow_st_dict[ticker][col] = cashflow_st_dict[ticker][col].str.replace(r'[-|, ]', '', regex=True)
        cashflow_st_dict[ticker][col] = pd.to_numeric(cashflow_st_dict[ticker][col]) 
        print('cashflow_st_dict done')
        
        if col!="TTM": #yahoo has ttm column for income statement and cashflow statement only
            balance_sheet_dict[ticker][col] = balance_sheet_dict[ticker][col].str.replace(r'[-|, ]', '', regex=True)
            balance_sheet_dict[ticker][col] = pd.to_numeric(balance_sheet_dict[ticker][col])
            print('balance_sheet_dict done')

    #     break
    # break

AAPL TTM
income_statatement_dict done
cashflow_st_dict done
AAPL 9/30/2023
income_statatement_dict done
cashflow_st_dict done
balance_sheet_dict done
AAPL 9/30/2022
income_statatement_dict done
cashflow_st_dict done
balance_sheet_dict done
AAPL 9/30/2021
income_statatement_dict done
cashflow_st_dict done
balance_sheet_dict done
AAPL 9/30/2020
income_statatement_dict done
cashflow_st_dict done
balance_sheet_dict done
CSCO TTM
income_statatement_dict done
cashflow_st_dict done
CSCO 7/31/2023
income_statatement_dict done
cashflow_st_dict done
balance_sheet_dict done
CSCO 7/31/2022
income_statatement_dict done
cashflow_st_dict done
balance_sheet_dict done
CSCO 7/31/2021
income_statatement_dict done
cashflow_st_dict done
balance_sheet_dict done
CSCO 7/31/2020
income_statatement_dict done
cashflow_st_dict done
balance_sheet_dict done
INFY.NS TTM
income_statatement_dict done
cashflow_st_dict done
INFY.NS 3/31/2024
income_statatement_dict done
cashflow_st_dict done
balance_sheet_dict done
INFY

In [223]:
income_statatement_dict[tickers[3]]

,TTM,12/31/2023,12/31/2022,12/31/2021,12/31/2020,12/31/2019
Total Revenue,6.166170e+08,6.219240e+08,5.847220e+08,6.040810e+08,5.653730e+08,NaN
Income from Associates & Other Participating Interests,1.277000e+06,9.650000e+05,6.450000e+05,1.478000e+06,1.580000e+05,NaN
Special Income Charges,6.100000e+04,6.800000e+04,5.966000e+06,1.141000e+06,6.350000e+05,NaN
Other Non Operating Income Expenses,1.804000e+06,1.535000e+06,1.370000e+05,4.890000e+05,1.254000e+06,NaN
Pretax Income,2.882240e+08,2.956080e+08,2.836410e+08,2.766200e+08,2.463780e+08,NaN
Tax Provision,4.360500e+07,4.923700e+07,4.691600e+07,4.928100e+07,4.128200e+07,NaN
Net Income Common Stockholders,2.302300e+08,2.319040e+08,2.265220e+08,2.165590e+08,1.928700e+08,NaN
Basic EPS,7.300000e-01,7.400000e-01,7.200000e-01,7.000000e-01,6.100000e-01,NaN
Diluted EPS,7.300000e-01,7.400000e-01,7.200000e-01,7.000000e-01,6.100000e-01,NaN
Basic Average Shares,3.152739e+08,3.133838e+08,3.146139e+08,3.093700e+08,3.161803e+08,NaN


In [140]:
temp = pd.DataFrame(income_statement).T
temp.columns = table_title["Breakdown"]
income_statatement_dict[ticker] = temp     

In [141]:
temp

,TTM,9/30/2023,9/30/2022,9/30/2021,9/30/2020
Total Revenue,"381,623,000","383,285,000","394,328,000","365,817,000","274,515,000"
Cost of Revenue,"207,657,000","214,137,000","223,546,000","212,981,000","169,559,000"
Gross Profit,"173,966,000","169,148,000","170,782,000","152,836,000","104,956,000"
Operating Expense,"55,726,000","54,847,000","51,345,000","43,887,000","38,668,000"
Operating Income,"118,240,000","114,301,000","119,437,000","108,949,000","66,288,000"
Net Non Operating Interest Income Expense,--,"-183,000","-106,000","198,000","890,000"
Other Income Expense,"55,000","-382,000","-228,000","60,000","-87,000"
Pretax Income,"118,112,000","113,736,000","119,103,000","109,207,000","67,091,000"
Tax Provision,"17,723,000","16,741,000","19,300,000","14,527,000","9,680,000"
Net Income Common Stockholders,"100,389,000","96,995,000","99,803,000","94,680,000","57,411,000"


In [137]:
pd.DataFrame(income_statement).T

,0,1,2,3,4
Total Revenue,"381,623,000","383,285,000","394,328,000","365,817,000","274,515,000"
Cost of Revenue,"207,657,000","214,137,000","223,546,000","212,981,000","169,559,000"
Gross Profit,"173,966,000","169,148,000","170,782,000","152,836,000","104,956,000"
Operating Expense,"55,726,000","54,847,000","51,345,000","43,887,000","38,668,000"
Operating Income,"118,240,000","114,301,000","119,437,000","108,949,000","66,288,000"
Net Non Operating Interest Income Expense,--,"-183,000","-106,000","198,000","890,000"
Other Income Expense,"55,000","-382,000","-228,000","60,000","-87,000"
Pretax Income,"118,112,000","113,736,000","119,103,000","109,207,000","67,091,000"
Tax Provision,"17,723,000","16,741,000","19,300,000","14,527,000","9,680,000"
Net Income Common Stockholders,"100,389,000","96,995,000","99,803,000","94,680,000","57,411,000"


In [146]:
income_statement

{}

In [147]:
table_title


{}

In [48]:
len(tabl)

1

In [69]:
income_statement = {}
table_title = {}


tabl = soup.find_all("div" , {"class" : "table yf-1pgoo1f"})
for t in tabl:
    heading = t.find_all("div" , {"class": "tableHeader yf-1pgoo1f"})
    for top_row in heading:
        table_title[top_row.get_text(separator="|").split("|")[0]] = top_row.get_text(separator="|").split("|")[1:]
    rows = t.find_all("div" , {"class": "row lv-0 yf-1xjz32c"})
    for row in rows:
        income_statement[row.get_text(separator="|").split("|")[2].strip()] = row.get_text(separator="|").split("|")[3:]

        break

    
    break

table_title



{'': [' ',
  '381,623,000 ',
  '383,285,000 ',
  '394,328,000 ',
  '365,817,000 ',
  '274,515,000 ']}

In [57]:
row.get_text(separator="|")

' | |Total Revenue| |381,623,000 |383,285,000 |394,328,000 |365,817,000 |274,515,000 '

In [16]:
requests.get('https://finance.yahoo.com/quote/AAPL/financials/?p=AAPL')

<Response [404]>

In [7]:
from bs4 import BeautifulSoup
